# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import json
import pickle
import nltk
import tensorflow as tf
from tensorflow.keras.models import load_model
from preprocessing import text_preprocessing
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to C:\Users\Nadia
[nltk_data]     Nabilla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Nadia
[nltk_data]     Nabilla\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Load Model

In [3]:
with open('class_labels.pkl', 'rb') as file1:
  class_labels = pickle.load(file1)

with open('stopwords.txt', 'r') as sw:
    stopwords = json.load(sw)

lstm = load_model('lstm')

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 383)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 383, 128)          1236992   
                                                                 
 bidirectional (Bidirection  (None, 383, 64)           41216     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 383, 64)           0         
                                                                 
 batch_normalization (Batch  (None, 383, 64)           256       
 Normalization)                                                  
                                                        

# New Data

In [6]:
dataInf = {'review' : ['i love this app, the interface is so amazing',
                       'the app is getting better these days. i can access so many useful features!',
                       'this app sucks, just a copy of twitter']}

dataInf = pd.DataFrame(dataInf)
dataInf

,review
0,"i love this app, the interface is so amazing"
1,the app is getting better these days. i can ac...
2,"this app sucks, just a copy of twitter"


# Preprocessing

In [7]:
dataInf['review'] = dataInf.apply(lambda row: text_preprocessing(row['review'], stopwords, lemmatizer), axis=1)
dataInf

,review
0,love app interface amazing
1,app day access feature
2,app suck copy twitter


# Prediction

In [8]:
yPred_proba = lstm.predict(dataInf)
yPred_indices = np.argmax(yPred_proba, axis = -1)
yPred_labels = [class_labels[idx] for idx in yPred_indices]
dataInf['rating'] = yPred_labels
dataInf

1/1 [==============================] - 1s 981ms/step


,review,rating
0,love app interface amazing,0
1,app day access feature,0
2,app suck copy twitter,1


# Kesimpulan

Berdasarkan hasil prediksi di atas, dapat dilihat bahwa ulasan yang diberikan oleh user pada index ke-0 dan ke-1 menunjukkan kesan netral terhadap aplikasi threads, sedangkan ulasan yang diberikan oleh user pada index ke-2 menunjukkan kesan positif, yang mana hasil prediksi pada index ke 0, 1, dan 2 tidak sesuai. Hal ini mungkin terjadi karena nilai accuracy dari model yang digunakan tidak cukup baik, yaitu 76.75%.